https://womantalks.ru/forum/67-hobbi/

In [1]:
# pip install bs

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm

df = pd.DataFrame(columns=['text'])
df

,text


In [2]:
url = 'https://womantalks.ru/forum/67-hobbi/'

In [3]:
response = requests.get(url)
print(response)

<Response [200]>


In [4]:
html = response.text

In [5]:
# bs = BeautifulSoup(response.text,"lxml")
# print(bs)

In [6]:
import re

In [7]:
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            print(q)
            if link in q['href']:
                links.append(q['href'])
    return sorted(list(set(links)))
get_links(get_html(url), url)

NameError: name 'get_html' is not defined

In [10]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import json

def get_html(url):
    response = requests.get(url)
    return response.text

def get_all_links(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_large ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            links.append(q['href'])
    return links

# def get_all_links_on_page(html, link):
#     soup = BeautifulSoup(html, features="html.parser")
#     tds = soup.find_all('tr')
#     for td in tds:
#         i = td.find('td', {'class':'td-block'})
#         if i != None:
#             return(i.find_all('a')[-1].text)
    
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if link in q['href']:
                links.append(q['href'])
    if len(links) == 0:
        return [link]
    return sorted(list(set(links)))

def get_all_links_one_theme(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if 'topic' in q['href'] and \
            'https://womantalks.ru/topic/1517-turizm/' not in q['href'] and \
            'https://womantalks.ru/topic/102-veloturizm/' not in q['href']:
                links.append(q['href'])
    return links

def get_page_data(html):
    soup = BeautifulSoup(urlopen(str(html.encode('utf-8'))[2:-1]), features="html.parser")

    # user
    tds = soup.find_all('strong', 'itemprop'=="name")
    users = []
    for td in tds:
        if 'Изменено' not in td.text:
            users.append(td.text)
#     print(users)


    # time
    tds = soup.find_all('p', {'class':"ipsType_reset"})
    times = []
    for td in tds:
        if 'Опубликовано' in td.text:
            times.append(td.text)
#     print(times)


    # text
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    tex = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find_all('p')
            tex.append(p[-1].text)
        else:
            tex.append(td.text)
#     print(tex)
    

    # quote
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    temp = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find('div', {'class':"ipsQuote_contents"})
            if p is not None:
                temp.append(p.text)
        else:
            temp.append('')
#     print(temp)

    data = ({'user': users,                    
             'time': times,      
             'text': tex,            
             'quote': temp,          
             'topic': ['Art&Hobbies'] * len(users)
            })          
    return users, times, tex, temp, ['Art&Hobbies'] * len(users)#data

def page_content(all_links, table):
#     for link in tqdm(all_links):
#     links = get_links(get_html(all_links), all_links)
#     for l in tqdm(all_links):
    all_links_one_theme = get_all_links_one_theme(get_html(all_links))
    for t in tqdm(all_links_one_theme):
#         print(t)
            page = get_links(get_html(t), t)
            for p in page:
#                 print(p)
                us, ti, te, tem, l = get_page_data(p)
                for i in range(len(us)):
                    if i > 5:
                        break
                    users.append(us[i])
                    times.append(ti[i])
                    tex.append(te[i])
                    temp.append(tem[i])
                    labels.append(l[i])
#                 table.append(data)

table = pd.DataFrame(columns={'users', 'times', 'text', 'quote', 'topic'})
users = []
times = []
tex = []
temp = []
labels = []
    
url = 'https://womantalks.ru/forum/67-hobbi/'

# all_links = get_links(get_html(url), url)
# print(all_links)
page_content(url, table)

table.loc[:, 'users'] = users
table.loc[:, 'times'] = times
table.loc[:, 'text'] = tex
table.loc[:, 'quote'] = temp
table.loc[:, 'topic'] = labels

0it [00:00, ?it/s]

In [9]:
pd.DataFrame(table).to_csv('Hobby_woman_talks_Copy.csv', index=False)

In [6]:
D = pd.read_csv('Hobby_woman_talks_Copy.csv')
D

,user,time,text,quote,topic
0,"[['\nsaeva6'], ['\nЕкатерина Дятлова'], ['\nsa...","[['\nОпубликовано: 18 авг 2019\n'], ['\nОпубли...","[['\n\n\tЛюбимый учился в\xa0 экономическом, с...","[[''], [''], ['\n\n\t\t\tМне видео, очень понр...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
1,"[['\nCvetokOK'], ['\nXyhuvupu'], ['\nNatali Da...","[['\nОпубликовано: 26 мар 2017\n'], ['\nОпубли...",[['\n\n\tПривет девочки\xa0\n\n\tВ данный моме...,"[[''], [''], [''], [''], [''], [''], [''], [''...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
2,"[['\nLapochka'], ['\nRealmccoy'], ['\nKaterina...","[['\nОпубликовано: 31 окт 2017\n'], ['\nОпубли...",[['\n\n\tКто нибудь делает ставки на спорт в и...,"[[''], [''], [''], [''], [''], [''], ['']]","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
3,[['\nLedivinter']],[['\nОпубликовано: 7 окт 2019\n']],[['\n\n\tПриветствую вас девченки.\xa0\n\tМое ...,[['']],['Art&Hobbies']
4,"[['\nГость'], ['\npewendutt']]","[['\nОпубликовано: 28 мар 2016\n'], ['\nОпубли...",[['\n\n\tШкола класс! Очень доволен что нашел ...,"[[''], ['']]","['Art&Hobbies', 'Art&Hobbies']"
5,"[['\nКлетка'], ['\nГрета'], ['\nXuos'], ['\nXy...","[['\nОпубликовано: 2 июн 2016\n'], ['\nОпублик...",[['\n\n\tВ последнее время стали популярны укр...,"[[''], [''], [''], [''], [''], [''], [''], [''...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
6,"[['\nОльгаФокс'], ['\nRenata'], ['\nSvetikthyn...",[['\nОпубликовано: 14 фев 2019\n\t\t\t\n\t\t\t...,[['\n\n\tКоллекционирую столовое серебро ручно...,"[[''], [''], ['']]","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies']"
7,"[['\nDasha'], ['\nАнна Викторовна'], ['\nМайск...","[['\nОпубликовано: 3 фев 2016\n'], ['\nОпублик...",[['\n\n\tЯ в последнее время начала много вяза...,"[[''], [''], [''], [''], [''], [''], [''], ['\...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
8,"[['\nlada_grom'], ['\nResha90'], ['\nlada_grom...","[['\nОпубликовано: 5 дек 2018\n'], ['\nОпублик...","[['\n\tжелаю успехов)))\n'], ['\n\tСпасибо, оч...","[['\n\n\t\t\tНадеюсь.\n\t\t\n'], ['\n\n\t\t\tж...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
9,"[['\nГость'], ['\nwomantalks.ru'], ['\nCocoNat...",[['\nОпубликовано: 5 ноя 2017\n\t\t\t\n\t\t\t\...,"[['\n\r\n\tНаверно у каждой девушки, ну или по...","[[''], [''], [''], ['\n\n\t\t\tДобро пожаловат...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."


In [7]:
count = 0
for i in range(len(D)):
    count += len(D['user'][i].split())
print('Количество записей с одного сайта по одной теме:', count)

Количество записей с одного сайта по одной теме: 345


In [15]:
import random
import numpy as np
ind = np.random.choice(len(D), 23, False)
ind

array([19, 20,  4, 18,  1, 22, 13, 14,  9, 15,  6, 12,  3, 16, 23, 11, 10,
        7, 17,  5,  2,  0, 21])

In [16]:
X = []
for i in ind:
    X.append(np.random.choice(D['text'][i].split(sep=']'), 3, False))

In [39]:
X[22]

array(['',
       ", ['\\n\\n\\tя задумывалась о лове-фотосессии) было бы прикольно)\\xa0\\n\\n'",
       "[['\\n\\n\\tНикогда не делала\\xa0фото для инстаграмма, но прочитала очень полезную статью и думаю, что начну делать снимки. Только я хочу сделать что-то необычное и интересно. Чтобы не просто были фотографии, а целая история, которую не стыдно показать людям. Моя подруга уже использовала советы из статьи, и у нее получилось показать свой стильный внешний вид. Кто уже занимался таким делом? Насколько оно вас заинтересовало?\\xa0\\n\\n'"],
      dtype='<U432')

In [40]:
3+2+1+3+2+0+2+3+3+0+1+0+1+1+0+1+1+1+1+2+2+3+2

35

In [42]:
(35/69)*(23*3)/100

0.35

In [44]:
35/(69-23)

0.7608695652173914

In [3]:
D = pd.read_csv('Hobby_woman_talks.csv')
D

,user,time,text,quote,topic
0,"['\nsaeva6', '\nЕкатерина Дятлова', '\nsaeva6'...","['\nОпубликовано: 18 авг 2019\n', '\nОпубликов...","['\n\n\tЛюбимый учился в\xa0 экономическом, со...","['', '', '\n\n\t\t\tМне видео, очень понравило...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
1,"['\nCvetokOK', '\nXyhuvupu', '\nNatali Danilen...","['\nОпубликовано: 26 мар 2017\n', '\nОпубликов...",['\n\n\tПривет девочки\xa0\n\n\tВ данный момен...,"['', '', '', '', '', '', '', '', '', '', '', '...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
2,"['\nLapochka', '\nRealmccoy', '\nKaterina_M', ...","['\nОпубликовано: 31 окт 2017\n', '\nОпубликов...",['\n\n\tКто нибудь делает ставки на спорт в ин...,"['', '', '', '', '', '', '']","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
3,['\nLedivinter'],['\nОпубликовано: 7 окт 2019\n'],['\n\n\tПриветствую вас девченки.\xa0\n\tМое л...,[''],['Art&Hobbies']
4,"['\nГость', '\npewendutt']","['\nОпубликовано: 28 мар 2016\n', '\nОпубликов...",['\n\n\tШкола класс! Очень доволен что нашел т...,"['', '']","['Art&Hobbies', 'Art&Hobbies']"
5,"['\nКлетка', '\nГрета', '\nXuos', '\nXyhuvupu'...","['\nОпубликовано: 2 июн 2016\n', '\nОпубликова...",['\n\n\tВ последнее время стали популярны укра...,"['', '', '', '', '', '', '', '', '', '']","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
6,"['\nОльгаФокс', '\nRenata', '\nSvetikthync']",['\nОпубликовано: 14 фев 2019\n\t\t\t\n\t\t\t\...,['\n\n\tКоллекционирую столовое серебро ручной...,"['', '', '']","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies']"
7,"['\nDasha', '\nАнна Викторовна', '\nМайская', ...","['\nОпубликовано: 3 фев 2016\n', '\nОпубликова...",['\n\n\tЯ в последнее время начала много вязат...,"['', '', '', '', '', '', '', '\n\n\t\t\tно даж...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
8,"['\nlada_grom', '\nResha90', '\nlada_grom', '\...","['\nОпубликовано: 5 дек 2018\n', '\nОпубликова...","['\n\tжелаю успехов)))\n', '\n\tСпасибо, очень...","['\n\n\t\t\tНадеюсь.\n\t\t\n', '\n\n\t\t\tжела...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."
9,"['\nГость', '\nwomantalks.ru', '\nCocoNatka', ...",['\nОпубликовано: 5 ноя 2017\n\t\t\t\n\t\t\t\t...,"['\n\r\n\tНаверно у каждой девушки, ну или поч...","['', '', '', '\n\n\t\t\tДобро пожаловать на фо...","['Art&Hobbies', 'Art&Hobbies', 'Art&Hobbies', ..."


In [4]:
count = 0
for i in range(len(D)):
    count += len(D['user'][i].split())
print('Количество записей с одного сайта по одной теме:', count)

Количество записей с одного сайта по одной теме: 345
